In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

**Types of recomendation system**:
1. **Content based** --> based on the content it will recomend(eg: if you watch any video in youtube you will get the similer video next time)
2. **Collabrative filter based** --> based on the user behavoir (eg : if their are two persons x and y if x and y see the same movie on amazon or any ott and they used to give same rating ,if next time the x person see any other movie it will recomend the person y to see it)--->(best example instagram,facebook,linkedin) 
3. **hybrid based** --> compination of top two is hybrid

**HERE WE USE THE CONTENT BASED**

In [2]:
movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")
credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")

In [3]:
movies.head(3)

In [4]:
movies.shape

In [5]:
credits.head(3)

In [6]:
## we can combine this two dataset by using the similar in it we can see their are two columns which are similar movie_id,title
pd.set_option("display.max_columns",None)
data = movies.merge(credits,on="title")
data.head(2)

In [7]:
# Not important columns

# budget -- >budget is not so important
# homepage -- > you can see any where not only in same website
# id -- > 
# original_language -- > the data is skewed because most of the language movies are en 
# original_title -- > i will drp this and i will keep title (because in this their will be orginal name of local language name of the movie)
# popularity -- > i guess it not more important because 50 % of the people may watch some movies on like their genres but not on their so popular 
# production_comapny -- > it not that important as who ever it is produced we will see those movies 
# production_countries -- > because now a days people are seeing some indian movies and some are seeing some web series of different countries like money hiest,dark
# release-data -- > i guess it is not so important

data_imp_columns = data[['movie_id','title','overview','genres','keywords','cast','crew']]
data_imp_columns.head(2)

In [8]:
## lets see any nan values
data_imp_columns.isna().sum()

In [9]:
data_imp_columns[data_imp_columns["overview"].isna()]

In [10]:
data_imp_columns.shape

In [11]:
data_imp_columns.dropna(inplace=True)
data_imp_columns.reset_index(inplace=True)
data_imp_columns.shape

In [12]:
data_imp_columns[data_imp_columns.duplicated()]

In [13]:
data_imp_columns.iloc[0]["genres"]
# i want this in this format [Action,Adventure,Fantasy,SciencE fICTION]

In [14]:
# f = data_imp_columns.iloc[0]["genres"]
# s = []
# for i in f:
#     s.append(i['name'])
# print(s)


################***ERROR**####################

# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# /tmp/ipykernel_33/4098515969.py in <module>
#       2 s = []
#       3 for i in f:
# ----> 4     s.append(i['name'])
#       5 print(s)

# TypeError: string indices must be integers

**SEE WE GOT AN ERROR BECAUSE THE VALUE OF F IS IN THE STINGS NOT IN THE LIST IT IS IN THE FORMAT OF STRINGS OF LIST OD DICT**

**SOLUTION**:

**SO WE HAVE TO CONVCERT IT INTO LIST**
their is module in python it is ase

In [15]:
import ast
f = data_imp_columns.iloc[0]["genres"]
d = ast.literal_eval(f)
print(d) ## it got change to list right

In [16]:
def convert(text):
    val = []
    for index in ast.literal_eval(text):
        val.append(index['name'])
    return val        

In [17]:
convert(f)

In [18]:
data_imp_columns.columns

In [19]:
data_imp_columns["genres"] = data_imp_columns["genres"].apply(convert)
data_imp_columns.head(1)

In [20]:
data_imp_columns["keywords"] = data_imp_columns["keywords"].apply(convert)
data_imp_columns.head(1)

In [21]:
data_imp_columns.iloc[0]["cast"] ## picking the name of first 3 dictionary because most of the people says the first 3 are the fav actor in that movie

In [22]:
def convert_cast(text):    ## picking top three cast names
    val = []
    count = 0 
    for index in ast.literal_eval(text):
        if count < 3:
            val.append(index['name'])
        count+=1
    return val      

In [23]:
data_imp_columns["cast"] = data_imp_columns["cast"].apply(convert_cast)
data_imp_columns.head(1)

In [24]:
data_imp_columns['crew'][0]

In [25]:
def convert_director(text):
    val = []
    for index in ast.literal_eval(text):
        if index["job"] == "Director":
            val.append(index["name"])
    return val         

In [26]:
data_imp_columns["crew"] = data_imp_columns["crew"].apply(convert_director)
data_imp_columns.head(2)

In [27]:
data_imp_columns["overview"][0]

In [28]:
data_imp_columns["overview"] = data_imp_columns["overview"].apply(lambda x:x.split())
data_imp_columns.head(2)

In [29]:
d = 'Sam Worthington'
d.replace(" ","")

In [30]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [31]:
# # genres,keywords,cast,crew
# data_imp_columns["genres"] = data_imp_columns["genres"].apply(collapse)
# data_imp_columns["keywords"] = data_imp_columns["keywords"].apply(collapse)
# data_imp_columns["cast"] = data_imp_columns["cast"].apply(collapse)
# data_imp_columns["crew"] = data_imp_columns["crew"].apply(collapse)
# data_imp_columns.head()

In [32]:
# data_imp_columns.head(2)

**YOU CAN DO THIS WITH THE ABOVE FUNCTION OR ELSE WITH THE LAMBDA KEY WORD**

In [33]:
data_imp_columns["genres"] = data_imp_columns["genres"].apply(lambda x:[i.replace(' ','') for i in x])
data_imp_columns["keywords"] = data_imp_columns["keywords"].apply(lambda x:[i.replace(' ','') for i in x])
data_imp_columns["cast"] = data_imp_columns["cast"].apply(lambda x:[i.replace(' ','') for i in x])
data_imp_columns["crew"] = data_imp_columns["crew"].apply(lambda x:[i.replace(' ','') for i in x])

In [34]:
data_imp_columns.head(2)

In [35]:
# overview	genres	keywords	cast	crew
data_imp_columns["total_overview"] = data_imp_columns["overview"] + data_imp_columns["genres"] + data_imp_columns["keywords"] + data_imp_columns["cast"] + data_imp_columns["crew"]

In [36]:
total_data = data_imp_columns.drop(["overview","genres","keywords","cast","crew"],axis=1)
total_data.head()

In [37]:
total_data["total_overview"] = total_data["total_overview"].apply(lambda x:" ".join(x))
total_data.head()

In [38]:
g = ["mosh ","mishra"]
d = " ".join(g)
print(d)

In [39]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidfv = TfidfVectorizer(max_features=5000,stop_words="english")

In [40]:
# vector = tfidfv.fit_transform(total_data["total_overview"]).toarray()
# vector[5]

In [41]:
# vector.shape

**WE HAVE LOT OF SIMILAR WORDS LIKE ['visit','visited','visits'] SO I THOUGH STEEMING IS GOOD PROCESS**

**STEMMING HAS TWO TYPES**
1. LEMITIATION -- >  TAKES MORE TIME  -- > THIS IS BEST
2. STEMIING ---> TAKES LESS TIME -- > THIS IS GOOD FOR SMALL DATA NOT FOR LARGE DATA

In [42]:
# ['visit','visited','visits'] ---- > words got duplicated
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()
for i in range(len(total_data["total_overview"])):
    review = re.sub("[^a-zA-Z]"," ",total_data["total_overview"][i])
    review = review.split()
#     review = review.lower()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    total_data["total_overview"][i] = " ".join(review)
total_data.head(2)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(max_features=5000,stop_words="english")

In [44]:
vector = tfidfv.fit_transform(total_data["total_overview"]).toarray()
vector[5]

In [45]:
total_data.iloc[2662]["total_overview"]

In [46]:
vector.shape

In [47]:
tfidfv.get_feature_names()[:20]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
smililarity = cosine_similarity(vector)
smililarity

In [ ]:
# won,zebra
smililarity.shape

**NOW EVERYTHING IS SETUP NOW WE GUYS HAVE TO MAKE A FUNCTION IF YOU TYPE THAT IN SEARCH IT SHOULD RECOMEND YOU THE SIMILAR MOVIES LIKE 10 SIMILAR MOVIES IS GOOD I GUESS**

In [ ]:
total_data[total_data['title'] == 'The Dark Knight Rises']

In [ ]:
total_data[total_data['title'] == 'The Dark Knight Rises'].index[0]

In [ ]:
smililarity[0]

In [ ]:
L = ["cccc", "b", "dd", "aaa"]
 
print("Normal sort :", sorted(L))
 
print("Sort with len :", sorted(L,reverse=True,key=len))

**BELOW CODE EXPLAIN :**
1. similarities[0] we get the all the values of it take those values 
2. you need the index for those values so we have done enumerate 
3. you usually dosen't get the output of enumetae so we put values into a list
4. sortting the values based of reverse order 
5. by using which we have to reverse the key using the values we got from similaritires so this is the distance of top 10 movies 

**IF YOU GUYS DIDN'T GET THIS CODE PLEASE DO WATCH THE ABOVE TWO CELLS**

In [ ]:
f = sorted(list(enumerate(smililarity[total_data[total_data['title'] == 'The Dark Knight Rises'].index[0]])),reverse=True,key=lambda x:x[1]) 
s = f[:10]
s

In [ ]:
def recommand(movie):
    index_value = total_data[total_data['title'] == movie].index[0]
    distance_movies = sorted(list(enumerate(smililarity[total_data[total_data['title'] == movie].index[0]])),reverse=True,key=lambda x:x[1])
    for each_movie in distance_movies[1:11]:
#         print(each_movie[0])
        print(total_data.iloc[each_movie[0]]["title"])

In [ ]:
recommand("The Dark Knight Rises")

In [ ]:
recommand("Pirates of the Caribbean: At World's End")

In [ ]:
import pickle
pickle.dump()